<img src=https://raw.githubusercontent.com/daftcode/daftacademy-python_levelup-spring2021/master/background.png>

![Plan kursu](https://raw.githubusercontent.com/daftcode/daftacademy-python_levelup-spring2021/master/program.png)

# A jak Asyncio


### Paweł Koszałka
### Daft Academy Python level up 2021
### 
### 19.05.2021

# Plan wykładu

1. Czym jest asyncio?
2. Programowanie współbieżne
3. Jak działa asyncio?
4. Jak używać asyncio?
5. Asynchroniczność w sieci.
6. Pytania
7. Linki

# 1. Czym jest asyncio?

- biblioteką standardową umożliwiająca programowanie asynchroniczne

- reużyciem opracowanego konceptu - pętli zdarzeń (eventloop)

- ewolucją w świecie pythona

- biblioteką pozwalającą na realizacje współbieżności

# 2. Programowanie współbieżne

- wykonuje jedno zadanie na raz, może obsłużyć wiele zadań
- przeciwieństwo wykonania sekwencyjnego
- gambit współbieżności

# 2.1 Wykonanie sekwencyjne programu

```python
import requests

def get_current_temp(city_name):
    return requests.get(f"http://weather.service.com?cityName={city_name}")

warsaw_temp = get_current_temp("Warsaw")
radom_temp = get_current_temp("Radom")
lublin_temp = get_current_temp("Lublin")
```

In [ ]:
<warsaw_temp> ---I/O NET--> RESULT <radom_temp> ---I/O NET--> RESULT <lublin_temp> ---I/O NET--> RESULT

- I/O bound task: network, disk
- marnotrawstwo zasobu - 1s == 3 miliardy cykli 3 GHZ procesora == tysiące niskopoziomowych operacji == setki potencjalnych operacji w pythonie

# 2.2 Wykonanie współbieżne

In [ ]:
<warsaw_temp> ---I/O NET--> RESULT
              <radom_temp> ---I/O NET--> RESULT
                           <lublin_temp> ---I/O NET--> RESULT

- niskopoziomowe - procesy i wątki
- wysykopoziomowe - asyncio

# 2.3 Procesy

- uruchomiony program
- system operacyjny przydziela zasoby
- posiada co najmniej jeden wątek
- obiążenie zasobów

# 2.4 Wątki
- głowna część wykonawcza programu
- współdzieli zasoby z procesem - oprócz CPU
- potrzebują synchronizacji: Lock, Semaphore
- problematyczne: lock starvation, deadlock, zmiana kontekstu
- mniejsze obciążenie zasobów

# 2.5 Wątki w języku python

- tylko jeden wątek uruchomiony na raz
- GIL - lock związany z interpreterem pythona
- zwalniany w momencie: wykonania operacji I/O, wykorzystania przydzielonego czasu procesora, wykonania kodu bibliotek C


# 2.6 sekwencyjne vs współbieżne w pythonie

# 2.6.1 Przykład sekwencyjny

In [1]:
import time
import httpx

def call_httpbin(delay):
    resp = httpx.get(f"http://httpbin.org/delay/{delay}")
    return resp.status_code

def main():
    start_time = time.time()
    responses = [
        call_httpbin(1),
        call_httpbin(2),
        call_httpbin(3),
        call_httpbin(4),
    ]
    end_time = time.time() - start_time
    print(f"Received {responses} in {end_time}")

main()

Received [200, 200, 200, 200] in 11.395669221878052


# 2.6.2 Przykład współbieżny z wątkami

In [6]:
import time
import requests
import threading

In [8]:
responses_lock = threading.Lock()

def call_httpbin(responses, delay):
    resp = requests.get(f"http://httpbin.org/delay/{delay}")
    responses_lock.acquire()
    responses.append(resp.status_code)
    responses_lock.release()

def main():
    start_time, responses, call_threads = time.time(), [], []
    for i in range(1, 5):
        call_thread = threading.Thread(target=call_httpbin, args=(responses, i))
        call_threads.append(call_thread)
        call_thread.start()

    for element in call_threads:
        element.join()

    end_time = time.time() - start_time
    print(f"Received {responses} in {end_time}")

main()

Received [200, 200, 200, 200] in 4.236565113067627


# 3. Jak działa asyncio

- eventloop uruchomiony w głownym wątku
- operuje na couroutines
- potrafi zatrzymać i wznawiać prace nad coroutines
- kernel I/O async
- reaktywne i proaktywne
- sychnroniczne biblioteki psują działanie

# 3.1 coroutine (współprogram)

- specjalny rodzaj generatora
- generatory są funkcjami które tworzą swoje wartości w trakcie iteracji
- wykorzystywane wlaściwości: wstrzymywanie i wznawianie działania

# 3.1.1 generatory - metoda next

In [3]:
def count_to(max_val):
    counter = 1
    while counter <= max_val:
        yield counter
        counter += 1

count_to_three = count_to(3)
print(next(count_to_three))
print(next(count_to_three))
print(next(count_to_three))

1
2
3


# 3.1.2 generatory - metoda send

In [5]:
def count_to(max_val):
    counter = 1
    while counter <= max_val:
        i = (yield counter)
        if i is not None:
            counter = i
        else:
            counter += 1

count_to_three = count_to(3)
print("next method: ")
print(next(count_to_three))
count_to_three.send(2)
print("next method: ")
print(next(count_to_three))

next method: 
1
next method: 
3


# 3.1.3 generatory - zagłębianie

In [14]:
def another_count():
    yield 4

def count_to(max_val):
    counter = 1
    while counter <= max_val:
        yield counter
        counter += 1
    yield from another_count()

print("Using for loop: ")
count_to_three = count_to(3)
for count in count_to_three:
    print(count)

Using for loop: 
1
2
3
4


# 3.2 Task

- tworzony automatycznie przez pętle
- recznie przez create_task
- interfejs do zarządzania: status, wyniki, obsługa

# 3.3 async def

In [ ]:
# Python 3.5+
async def coro_name(args):
    # your code here

# Python 3.4
@asyncio.coroutine
def coro_name(args):
    # your code here

# 3.4 await

In [ ]:
# Python 3.5+
async def coro_name(args):
    # your code here
    await another_coro()

# Python 3.4
@asyncio.coroutine
def coro_name(args):
    yield from another_coro()

# 4. Jak używać asyncio?

# 4.1 Uruchomienie korutyny

In [ ]:
import datetime
import asyncio

In [18]:
async def print_lab():
    print(f"{datetime.datetime.now()}: Daft")
    await asyncio.sleep(2)
    print(f"{datetime.datetime.now()}: Academy")

async def main():
    await print_lab()

# if __name__ == "__main__":
#     asyncio.run(main())

await main()

2021-05-19 10:43:57.400390: Daft
2021-05-19 10:43:59.401823: Academy


# 4.2 Uruchomienie współbieżne dwóch korutyn

In [ ]:
import datetime
import asyncio

In [22]:
async def print_lab(coro_id):
    print(f"{datetime.datetime.now()} {coro_id}: Daft")
    await asyncio.sleep(2)
    print(f"{datetime.datetime.now()} {coro_id}: Academy")

async def main():
    await print_lab("1")
    await print_lab("2")

await main()

2021-05-19 10:45:47.933421 1: Daft
2021-05-19 10:45:49.936396 1: Academy
2021-05-19 10:45:49.936934 2: Daft
2021-05-19 10:45:51.939775 2: Academy


# 4.2.1 Uruchomienie współbieżnie dwóch korutyn - wersja poprawna

In [ ]:
import datetime
import asyncio

In [14]:
async def print_lab(coro_id):
    print(f"{datetime.datetime.now()} {coro_id}: Daft")
    await asyncio.sleep(2)
    print(f"{datetime.datetime.now()} {coro_id}: Academy")

async def main():
    await asyncio.gather(print_lab("1"), print_lab("2"))

2021-05-19 10:29:41.775482 1: Daft
2021-05-19 10:29:41.776381 2: Daft
2021-05-19 10:29:43.778645 1: Academy
2021-05-19 10:29:43.778751 2: Academy


# 4.3 Obiekt Task

In [ ]:
import datetime
import asyncio

In [23]:
async def print_lab(coro_id):
    print(f"{datetime.datetime.now()} {coro_id}: Daft")
    await asyncio.sleep(2)
    print(f"{datetime.datetime.now()} {coro_id}: Academy")

async def main():
    print(f"{datetime.datetime.now()} Started main")
    task_1 = asyncio.create_task(print_lab("1"))
    task_2 = asyncio.create_task(print_lab("2"))
    print(f"{datetime.datetime.now()} Created tasks")

await main()

2021-05-19 10:47:15.015959 Started main
2021-05-19 10:47:15.016366 Created tasks
2021-05-19 10:47:15.016622 1: Daft
2021-05-19 10:47:15.016724 2: Daft
2021-05-19 10:47:17.019757 1: Academy
2021-05-19 10:47:17.020074 2: Academy


# 4.3.1 Obiekt Task - zatrzymanie

In [6]:
import datetime
import asyncio
import threading
import requests

In [26]:
async def print_lab(coro_id):
    print(f"{datetime.datetime.now()} {coro_id}: Daft")
    try:
        await asyncio.sleep(10)
    except asyncio.CancelledError:
        print(f"{datetime.datetime.now()} {coro_id}: Doing cleanup")
    else:
        print(f"{datetime.datetime.now()} {coro_id}: Academy")

async def main():
    task_1 = asyncio.create_task(print_lab("1"))
    await asyncio.sleep(4)
    print(f"{datetime.datetime.now()}: Cancel task_1")
    task_1.cancel()
    print(f"{datetime.datetime.now()}: Canceled task_1")

await main()

2021-05-19 10:48:11.801659 1: Daft
2021-05-19 10:48:15.803834: Cancel task_1
2021-05-19 10:48:15.804240: Canceled task_1
2021-05-19 10:48:15.804910 1: Doing cleanup


# 4.4 wykonanie sekwencyjne vs asyncio vs wątki

In [29]:
import time
import httpx

def call_httpbin(delay):
    resp = httpx.get(f"http://httpbin.org/delay/{delay}")
    return resp.status_code

def main():
    start_time = time.time()
    responses = [call_httpbin(1), call_httpbin(2), call_httpbin(3), call_httpbin(4)]
    end_time = time.time() - start_time
    print(f"Received {responses} in {end_time}")

main()

Received [200, 200, 200, 200] in 11.031611919403076


In [31]:
async def call_httpbin(delay):
    async with httpx.AsyncClient() as client:
        response = await client.get(f"http://httpbin.org/delay/{delay}")
        return response.status_code


async def main():
    start_time = time.time()
    responses = await asyncio.gather(call_httpbin(1), call_httpbin(2), call_httpbin(3), call_httpbin(4))
    end_time = time.time() - start_time
    print(f"Received {responses} in {end_time}")


await main()

Received [200, 200, 200, 200] in 4.372626066207886


In [7]:
responses_lock = threading.Lock()

def call_httpbin(responses, delay):
    resp = requests.get(f"http://httpbin.org/delay/{delay}")
    responses_lock.acquire()
    responses.append(resp.status_code)
    responses_lock.release()

def main():
    start_time, responses, call_threads = time.time(), [], []
    for i in range(1, 5):
        call_thread = threading.Thread(target=call_httpbin, args=(responses, i))
        call_threads.append(call_thread)
        call_thread.start()

    for element in call_threads:
        element.join()

    end_time = time.time() - start_time
    print(f"Received {responses} in {end_time}")

main()

Received [200, 200, 200, 200] in 4.314588785171509


# 4.5 Wątki vs asyncio

- pre-emptive multitasking
- cooperative multitasking

# 5. Asynchroniczność w sieci

```python

def validate_email(email):
    # validation logic, regexp, domain checking, blacklisting
    # raises InvalidEmail when email is invalid

@app.post("/validate_email")
def check_email(email: str):
    try:
        validate_email(email)
        User.objects.get(email=email) 
    except (InvalidEmail, User.DoesNotExist):
        return json({"valid": True})

    
@app.post("/async_validate_email")
async def check_email(email: str):
    try:
        validate_email(email)
        await User.objects.get(email=email) 
    except (InvalidEmail, User.DoesNotExist):
        return json({"valid": True})

```

# 6. Pytania?

# 7. Linki

- https://docs.python.org/3/library/asyncio.html

- https://realpython.com/python-concurrency/

- https://realpython.com/async-io-python/#the-asyncio-package-and-asyncawait

- https://www.youtube.com/watch?v=Xbl7XjFYsN4&list=PLhNSoGM2ik6SIkVGXWBwerucXjgP1rHmB